In [1]:
print('hello supervised learning')

hello supervised learning


In [2]:
import polars as pl

## Load 

Load from CSV and clean up the data
1. Remove all rows where the year is 2014 because its the first year there are no pbs
2. Clean up gender as it contains "M" or "Male" and "F" or "Female". Just take the first character for simplity
3. Data Types

In [37]:
results_with_pb_df = pl.read_csv("data/withpb-all-results.csv")

# Remove all rows where the year is 2014
results_with_pb_df = results_with_pb_df.filter(pl.col("year") != 2014)

results_with_pb_df = results_with_pb_df.drop("XC_Count_of_event", "XC_Position_Min", "XC_Position_Max", "XC_Min_Age", "XC_Max_Age","exists", "from_file")

# Gender Clean up
results_with_pb_df = results_with_pb_df.with_columns(
    pl.col("Gender").str.slice(0, 1).alias("Gender")
)

results_with_pb_df = results_with_pb_df.with_columns(
    pl.col("is_pb").cast(pl.Boolean),
    pl.col("first_time_run").cast(pl.Boolean)
)

display(results_with_pb_df)




Position,Time,Bib Number,Name,Gender,Gender Position,Club,Points,RaceName,year,month,day,date,distance,pace,club_position,club_position_gender,is_pb,first_time_run,prior_quickest_run_in_year,prior_quickest_run_time,prior_quickest_run_position,prior_time_vs_current_change,prior_time_vs_current_time,fellow_club_mates_running,fellow_club_mates_running_male,fellow_club_mates_running_female,best_position_year_to_date,average_position_year_to_date,worst_position_year_to_date,best_points_year_to_date,average_points_year_to_date,worst_points_year_to_date,runs_in_wsfrl_year_to_date,runs_in_wsfrl,years_in_wsfrl,year_with_best_position,year_with_worst_position,best_position_all_time,worst_position_all_time
i64,str,i64,str,str,f64,str,f64,str,i64,i64,i64,str,f64,str,f64,f64,bool,bool,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3,"""00:28:24""",328,"""Andy Massingale""","""M""",3.0,"""Henfield Joggers""",10.0,"""Beach Run""",2015,5,20,"""2015-05-20""",8.0,"""0:03:33""",1.0,1.0,false,true,null,"""nan""",null,null,null,15.0,9.0,6.0,3.0,8.0,20.0,10.0,10.0,10.0,4.0,4.0,2.0,2014.0,2014.0,3.0,20.0
4,"""00:28:24""",280,"""Jon Bess""","""M""",4.0,"""Worthing Striders""",10.0,"""Beach Run""",2015,5,20,"""2015-05-20""",8.0,"""0:03:33""",1.0,1.0,false,true,null,"""nan""",null,null,null,16.0,10.0,6.0,null,null,null,null,null,null,null,0.0,0.0,null,null,null,null
5,"""00:28:34""",461,"""Tiarnan Cotter""","""M""",5.0,"""Horsham Joggers""",10.0,"""Beach Run""",2015,5,20,"""2015-05-20""",8.0,"""0:03:34.250000""",1.0,1.0,false,true,null,"""nan""",null,null,null,33.0,14.0,19.0,15.0,19.0,23.0,10.0,10.0,10.0,2.0,2.0,2.0,2014.0,2015.0,15.0,23.0
6,"""00:28:50""",254,"""Paul Sargent""","""M""",6.0,"""Burgess Hill Runners""",10.0,"""Beach Run""",2015,5,20,"""2015-05-20""",8.0,"""0:03:36.250000""",1.0,1.0,false,true,null,"""nan""",null,null,null,47.0,28.0,19.0,6.0,7.0,8.0,10.0,10.0,10.0,2.0,3.0,2.0,2014.0,2014.0,6.0,54.0
7,"""00:29:11""",167,"""Paul Cousins""","""M""",7.0,"""Haywards Heath Harriers""",10.0,"""Beach Run""",2015,5,20,"""2015-05-20""",8.0,"""0:03:38.875000""",1.0,1.0,true,false,"""2014.0""","""00:29:34""","""4.0""","""up""","""00:23""",13.0,8.0,5.0,2.0,20.0,132.0,10.0,9.0,7.0,9.0,10.0,2.0,2015.0,2014.0,2.0,132.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
627,"""00:47:41""",631,"""Ann Savidge""","""F""",303.0,"""Burgess Hill Runners""",1.0,"""Worthing Lido""",2016,1,31,"""2016-01-31""",6.54,"""0:07:17.461774""",92.0,52.0,false,false,"""2015.0""","""00:38:16""","""571.0""","""down""","""09:25""",93.0,40.0,53.0,341.0,441.0,571.0,4.0,2.0,1.0,7.0,10.0,2.0,2014.0,2015.0,341.0,613.0
628,"""00:54:49""",613,"""Christine Jones""","""F""",304.0,"""Burgess Hill Runners""",1.0,"""Worthing Lido""",2016,1,31,"""2016-01-31""",6.54,"""0:08:22.905199""",93.0,53.0,false,true,null,"""nan""",null,null,null,93.0,40.0,53.0,null,null,null,null,null,null,null,0.0,0.0,null,null,null,null
629,"""00:56:27""",429,"""Kalpna Robertson""","""F""",305.0,"""Lewes AC""",1.0,"""Worthing Lido""",2016,1,31,"""2016-01-31""",6.54,"""0:08:37.889908""",34.0,14.0,false,true,null,"""nan""",null,null,null,33.0,20.0,13.0,null,null,null,null,null,null,null,0.0,0.0,null,null,null,null


## Find Categoricals

These are columns with limited subset of values.

For example education could be 
- Graduate
- Not Graduate

In our data 
- Club 
- RaceName
- Gender
All of which are unlikely to influence a pb

However, I'll add of these are categorical variables to the model.
- RaceName
- Gender




In [38]:
results_with_pb_with_dummies_df = results_with_pb_df.to_dummies(
    ["Gender", "RaceName"], drop_first=True
)

In [14]:
display(results_with_pb_with_dummies_df)

Position,Time,Bib Number,Name,Gender_F,Gender Position,Club,Points,RaceName_Downland Dash,RaceName_Fittleworth,RaceName_Great Walstead 5,RaceName_Gunpowder Trot,RaceName_Hangover-5,RaceName_Hedgehoppers 5,RaceName_Hickstead Gallop,RaceName_Highdown Hike,RaceName_Hornets Stinger,RaceName_Hove Park,RaceName_Lewes AC Easter Monday,RaceName_Run Your Heart Out 8k,RaceName_Seven Stiles,RaceName_Steepdown Challenge,RaceName_Steyning AC Roundhill Romp,RaceName_Tilgate 5,RaceName_Trundle View,RaceName_Valentine,RaceName_Windlesham-Whip,RaceName_Worthing Lido,year,month,day,date,exists,from_file,distance,pace,club_position,club_position_gender,is_pb,first_time_run,prior_quickest_run_in_year,prior_quickest_run_time,prior_quickest_run_position,prior_time_vs_current_change,prior_time_vs_current_time,fellow_club_mates_running,fellow_club_mates_running_male,fellow_club_mates_running_female,best_position_year_to_date,average_position_year_to_date,worst_position_year_to_date,best_points_year_to_date,average_points_year_to_date,worst_points_year_to_date,runs_in_wsfrl_year_to_date,runs_in_wsfrl,years_in_wsfrl,year_with_best_position,year_with_worst_position,best_position_all_time,worst_position_all_time,XC_Count_of_event,XC_Position_Min,XC_Position_Max,XC_Min_Age,XC_Max_Age
i64,str,i64,str,u8,f64,str,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i64,i64,i64,str,i64,str,f64,str,f64,f64,bool,bool,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3,"""00:28:24""",328,"""Andy Massingale""",0,3.0,"""Henfield Joggers""",10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",1,"""20150520-Beach Run""",8.0,"""0:03:33""",1.0,1.0,false,true,null,"""nan""",null,null,null,15.0,9.0,6.0,3.0,8.0,20.0,10.0,10.0,10.0,4.0,4.0,2.0,2014.0,2014.0,3.0,20.0,null,null,null,null,null
4,"""00:28:24""",280,"""Jon Bess""",0,4.0,"""Worthing Striders""",10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",1,"""20150520-Beach Run""",8.0,"""0:03:33""",1.0,1.0,false,true,null,"""nan""",null,null,null,16.0,10.0,6.0,null,null,null,null,null,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null
5,"""00:28:34""",461,"""Tiarnan Cotter""",0,5.0,"""Horsham Joggers""",10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",1,"""20150520-Beach Run""",8.0,"""0:03:34.250000""",1.0,1.0,false,true,null,"""nan""",null,null,null,33.0,14.0,19.0,15.0,19.0,23.0,10.0,10.0,10.0,2.0,2.0,2.0,2014.0,2015.0,15.0,23.0,null,null,null,null,null
6,"""00:28:50""",254,"""Paul Sargent""",0,6.0,"""Burgess Hill Runners""",10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",1,"""20150520-Beach Run""",8.0,"""0:03:36.250000""",1.0,1.0,false,true,null,"""nan""",null,null,null,47.0,28.0,19.0,6.0,7.0,8.0,10.0,10.0,10.0,2.0,3.0,2.0,2014.0,2014.0,6.0,54.0,null,null,null,null,null
7,"""00:29:11""",167,"""Paul Cousins""",0,7.0,"""Haywards Heath Harriers""",10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",1,"""20150520-Beach Run""",8.0,"""0:03:38.875000""",1.0,1.0,true,false,"""2014.0""","""00:29:34""","""4.0""","""up""","""00:23""",13.0,8.0,5.0,2.0,20.0,132.0,10.0,9.0,7.0,9.0,10.0,2.0,2015.0,2014.0,2.0,132.0,6.0,2.0,40.0,21.0,55.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
627,"""00:47:41""",631,"""Ann Savidge""",1,303.0,"""Burgess Hill Runners""",1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,1,31,"""2016-01-31""",1,"""20160131-Worthing Lido""",6.54,"""0:07:17.461774""",92.0,52.0,false,false,"""2015.0""","""00:38:16""","""571.0""","""down""","""09:25""",93.0,40.0,53.0,341.0,441.0,571.0,4.0,2.0,1.0,7.0,10.0,2.0,2014.0,2015.0,341.0,613.0,null,null,null,null,null
628,"""00:54:49""",613,"""Christine Jones""",1,304.0,"""Burgess Hill Runners""",1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,1,31,"""2016-01-31""",1,"""20160131-Worthing Lido""",6.54,"""0:08:22.90

## Split the data

Remove the target column from the features. **is_pb**

Features are the columns that will be used to predict the target variable. In this case, the features are all the columns except the is_pb column. 


In [42]:
pb_achieved_features_df = results_with_pb_with_dummies_df.drop( "is_pb")

# drop non feature columns 
pb_achieved_features_df = pb_achieved_features_df.drop("Bib Number", "Name", "date","Club" )
pb_achieved_features_df = pb_achieved_features_df.drop( "Time", "prior_quickest_run_time", "pace" )
pb_achieved_features_df = pb_achieved_features_df.drop( "prior_quickest_run_in_year", "prior_quickest_run_position","prior_time_vs_current_change", "prior_time_vs_current_time" )

display(pb_achieved_features_df)

pb_achieved_target_df = results_with_pb_df.select("is_pb")

Position,Gender_F,Gender Position,Points,RaceName_Downland Dash,RaceName_Fittleworth,RaceName_Great Walstead 5,RaceName_Gunpowder Trot,RaceName_Hangover-5,RaceName_Hedgehoppers 5,RaceName_Hickstead Gallop,RaceName_Highdown Hike,RaceName_Hornets Stinger,RaceName_Hove Park,RaceName_Lewes AC Easter Monday,RaceName_Run Your Heart Out 8k,RaceName_Seven Stiles,RaceName_Steepdown Challenge,RaceName_Steyning AC Roundhill Romp,RaceName_Tilgate 5,RaceName_Trundle View,RaceName_Valentine,RaceName_Windlesham-Whip,RaceName_Worthing Lido,year,month,day,distance,club_position,club_position_gender,first_time_run,fellow_club_mates_running,fellow_club_mates_running_male,fellow_club_mates_running_female,best_position_year_to_date,average_position_year_to_date,worst_position_year_to_date,best_points_year_to_date,average_points_year_to_date,worst_points_year_to_date,runs_in_wsfrl_year_to_date,runs_in_wsfrl,years_in_wsfrl,year_with_best_position,year_with_worst_position,best_position_all_time,worst_position_all_time
i64,u8,f64,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i64,i64,i64,f64,f64,f64,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3,0,3.0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,8.0,1.0,1.0,true,15.0,9.0,6.0,3.0,8.0,20.0,10.0,10.0,10.0,4.0,4.0,2.0,2014.0,2014.0,3.0,20.0
4,0,4.0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,8.0,1.0,1.0,true,16.0,10.0,6.0,null,null,null,null,null,null,null,0.0,0.0,null,null,null,null
5,0,5.0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,8.0,1.0,1.0,true,33.0,14.0,19.0,15.0,19.0,23.0,10.0,10.0,10.0,2.0,2.0,2.0,2014.0,2015.0,15.0,23.0
6,0,6.0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,8.0,1.0,1.0,true,47.0,28.0,19.0,6.0,7.0,8.0,10.0,10.0,10.0,2.0,3.0,2.0,2014.0,2014.0,6.0,54.0
7,0,7.0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,8.0,1.0,1.0,false,13.0,8.0,5.0,2.0,20.0,132.0,10.0,9.0,7.0,9.0,10.0,2.0,2015.0,2014.0,2.0,132.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
627,1,303.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,1,31,6.54,92.0,52.0,false,93.0,40.0,53.0,341.0,441.0,571.0,4.0,2.0,1.0,7.0,10.0,2.0,2014.0,2015.0,341.0,613.0
628,1,304.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,1,31,6.54,93.0,53.0,true,93.0,40.0,53.0,null,null,null,null,null,null,null,0.0,0.0,null,null,null,null
629,1,305.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,1,31,6.54,34.0,14.0,true,33.0,20.0,13.0,null,null,null,null,null,null,null,0.0,0.0,null,null,null,null


In [43]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

SEED = 123

(
    pb_achieved_feature_train_df,
    pb_achieved_feature_test_df,
    pb_achieved_target_train_df,
    pb_achieved_target_test_df,
) = train_test_split(
    pb_achieved_features_df,
    pb_achieved_target_df,
    test_size=0.30,
    random_state=SEED,
)



## Scale the data

It is important to scale the data so that the model is not distorted by really large values.

### My notes are 
- Dimensionality reduction, which 
1. Rotates the data. Some like 45 degrees
2. Keeps the same data but reduce the complexity
- Principal Component Analysis (PCA) is a technique that can be used to reduce the dimensionality of the data.

### Notes from Copilot
The StandardScaler standardizes a feature by subtracting the mean and then scaling to unit variance. Unit variance means dividing all the values by the standard deviation.

In [44]:
scaler = StandardScaler()

pb_achieved_feature_train_scaled_df = scaler.fit_transform(
    pb_achieved_feature_train_df
)

pb_achieved_feature_test_scaled_df = scaler.fit_transform(
    pb_achieved_feature_test_df
)